In [1]:
from pathlib import Path
import numpy as np
from funcionesTP import *

In [3]:
#Recibe la carpeta de donde conseguir los embeddings y cambiamos sus formatos para que sean matrices de NumPy
def cargarDataset(carpeta: Path):
    pathCats = carpeta.joinpath('cats/efficientnet_b3_embeddings.npy')
    pathDogs = carpeta.joinpath('dogs/efficientnet_b3_embeddings.npy')

    embeddingsCats = np.load(pathCats)
    embeddingsDogs = np.load(pathDogs)

    embeddings = np.concatenate((embeddingsCats, embeddingsDogs), axis=1)
    _, m = embeddings.shape

    Y = np.zeros((2,m))
    for i in range(0, int(m/2)):
        Y[0,i] = 1

    for i in range(int(m/2), m):
        Y[1,i] = 1

    return embeddings, Y

#Carga el dataset de los embeddings de entrenamiento y de evaluacion.
def cargarDatasetCompleto():
    Xt, Yt = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/train'))
    Xv, Yv = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))

    return Xt, Yt, Xv, Yv



Las siguientes funciones generan la matriz W a partir del calculo de la pseudoinversa de X utilizando diferentes metodos de decomposicion

In [3]:
def generar_W_EcuacionesNormales():
    Xt, Yt, Xv, Yv = cargarDatasetCompleto()

    W = pinvEcuacionesNormales(Xt, Yt)
    np.save('./W_eqnormales', W)
generar_W_EcuacionesNormales()

100%|██████████| 1536/1536 [03:05<00:00,  8.27it/s]


primer LU de LDV
iteracion LU


  2%|▏         | 29/1535 [01:58<1:42:54,  4.10s/it]


KeyboardInterrupt: 

In [ ]:
def generar_W_QRHH():
    Xt, Yt, Xv, Yv = cargarDatasetCompleto()

    Q, R = QR_con_HH(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrhh', W)
generar_W_QRHH()

100%|██████████| 2000/2000 [02:19<00:00, 14.35it/s]]


In [ ]:
def generar_W_QRGS():
    Xt, Yt, Xv, Yv = cargarDatasetCompleto()

    Q, R = QR_con_GS(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrgs', W)
generar_W_QRGS()

100%|██████████| 2000/2000 [26:16<00:00,  1.27it/s]


In [ ]:
def generar_W_SVD():
    Xt, Yt, Xv, Yv = cargarDatasetCompleto()
    svdFCN(Xt, Yt)
    # np.save('./W_SVD', W)
generar_W_SVD()

In [ ]:
# Recibe una matriz W  conseguida a traves de algunos de los metodos desarrollados y, utilizando la norma 2, calculamos la norma del residuo||Y - WX||_2 para tener una medida error sobre el conjunto de validacion.
def test_W(W):
    Xv, Yv = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))

    matrizDiferencia = Yv - (W @ Xv)
    print(np.linalg.norm(matrizDiferencia, ord=2))

Haciendo uso de la funcion `test_W()` las siguientes funciones calculan el error utilizando cada decomposicion para el calculo de W 

In [4]:
def generar_W_EcuacionesNormales():

    W = pinvEcuacionesNormales(Xt, Yt)
    np.save('./W_eqnormales.npy', W)
    
def testEqNormales():
    W = np.load('./W_eqnormales.npy')
    test_W(W)

generar_W_EcuacionesNormales()
testEqNormales()

NameError: name 'test_W' is not defined

In [ ]:
def generar_W_QRHH():

    Q, R = QR_con_HH(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrhh.npy', W)

def testQRHH():
    W = np.load('./W_qrhh.npy')
    test_W(W)

generar_W_QRHH()
testQRHH()

In [ ]:
def generar_W_QRGS():

    Q, R = QR_con_GS(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrgs', W)

def testQRGS():
    W = np.load('./W_qrgs.npy')
    test_W(W)
    
generar_W_QRGS()
testQRGS()

In [ ]:
def generar_W_SVD():
    W = svdFCN(Xt, Yt)
    np.save('./W_SVD.npy', W)

def testSVD():
    W = np.load('./W_SVD.npy')
    test_W(W, tol)

testCholesky()

In [7]:
# Evalúa el modelo contando cuántos gatos y perros fueron clasificados correctamente y cuántos incorrectamente.
def valoresDeConfusion(W_file):
    W = np.load(W_file)
    Xv, _ = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))
    matriz_resultado = W@Xv 
    gatos_positivos = 0

    for i in range(500):
        resultado = matriz_resultado[:,i]
        if resultado[0] > resultado[1]:
            gatos_positivos += 1 
    
    perros_positivos = 0

    for i in range(500, 1000):
        resultado = matriz_resultado[:,i]
        if resultado[1] > resultado[0]:
            perros_positivos += 1

    return gatos_positivos, 500 - gatos_positivos, perros_positivos, 500 - perros_positivos          

valoresDeConfusion('./W_qrgs.npy')

(334, 166, 350, 150)